# **PROJECT: EDA WALMART Sales Analysis**

##  **I. Project Objective & Value Proposition**
### **1.1 Objective:**
To conduct in-depth exploratory data analysis (EDA) on historical sales data from store CA_1 (M5 Forecasting dataset) to uncover **Sales (possibly demand)** patterns and generate actionable insights that support future forecasting and inventory optimization efforts.

### **1.2 Value proposition:**
- Inventory Insights: Identify key sales trends to support more efficient stock planning and reduce excess inventory.
- Operational Awareness: Reveal patterns useful for labor and space planning in warehouse operations.
- Proactive Stock Management: Surface insights that can help prevent stockouts and improve fulfillment reliability.
- Analytics Foundation: Demonstrates how EDA serves as a critical first step in applying data science to solve real-world supply chain challenges.


### **1.3 Data Description:**

- Sell prices dataset: Contains information about the price of the products sold per store and date
    + store_id: The id of the store where the product is sold.
    + item_id: The id of the product.
    + wm_yr_wk: The id of the week.
    + sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).

- Calendar dataset: Contains the dates on which products are sold. The dates are in a yyyy/dd/mm format
    + date: The date in a “y-m-d” format.
    + wm_yr_wk: The id of the week the date belongs to.
    + weekday: The type of the day (Saturday, Sunday, ..., Friday).
    + wday: The id of the weekday, starting from Saturday.
    + month: The month of the date.
    + year: The year of the date.
    + event_name_1: If the date includes an event, the name of this event.
    + event_type_1: If the date includes an event, the type of this event.
    + event_name_2: If the date includes a second event, the name of this event.
    + event_type_2: If the date includes a second event, the type of this event.
    + snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP 3 purchases on the examined date. 1 indicates that SNAP purchases are allowed.

- Sales data set: Contains the historical daily unit sales data per product and store [d_1 - d_1913]
    + item_id: The id of the product.
    + dept_id: The id of the department the product belongs to.
    + cat_id: The id of the category the product belongs to.
    + store_id: The id of the store where the product is sold.
    + state_id: The State where the store is located.
    + d_1, d_2, ..., d_i, ... d_1941: The number of units sold at day i, starting from 2011-01-29.

- Sample output dataset:
    Each row contains an id that is a concatenation of an item_id and a store_id, which is either validation (corresponding to the Public leaderboard), or evaluation (corresponding to the Private leaderboard). You are predicting 28 forecast days (F1-F28) of items sold for each row. For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

#### *Import neccessary libraries*

In [19]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from  datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## **II. Data Preparation**

#### *Load datasets*

In [14]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [15]:
def read_data(PATH):
    print('Reading files...')
    calendar = pd.read_csv(f'{PATH}/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv(f'{PATH}/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv(f'{PATH}/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv(f'{PATH}/sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission

calendar, sell_prices, sales, sample_output = read_data("/Users/apple/Downloads/")

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns


### **2.1 Initial Setup:**
I merge all dataframes (using pd.melt()) as a single dataframe for later EDA and Modeling.

In the original 4 files data from M5, 
- The products are sold across ten stores, located in three States (CA, TX and WI)
- For each State we have some Stores
- In each Store we have 3,049 products
- For each Product belongs to one of three Category are [Hobbies, Foods, Household]
- For each Category we have some Department
- For each Department we have some Products

For EDA I will focus on CA_1, which is one of the stores in California

In [16]:
sales_melt = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='sales')

In [5]:
sales_CA_1 = sales_melt[sales_melt.store_id == "CA_1"]
new_CA_1 = pd.merge(sales_CA_1, calendar, left_on="day", right_on="d", how="left")
new_CA_1 = pd.merge(new_CA_1, sell_prices, left_on=["store_id", "item_id", "wm_yr_wk"],right_on=["store_id", "item_id", "wm_yr_wk"], how="left")
new_CA_1["day_int"] = new_CA_1.day.apply(lambda x: int(x.split("_")[-1]))

# Drop unecessary columns
new_CA_1.drop(['d', 'day', 'id','store_id', 'wm_yr_wk', 'weekday','state_id', 'wday', 'month', 'year', 'snap_TX', 'snap_WI'], 
                  axis = 1, inplace = True)
new_CA_1.head()


,item_id,dept_id,cat_id,sales,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,day_int
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,0,2011-01-29,NaN,NaN,NaN,NaN,0,NaN,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,0,2011-01-29,NaN,NaN,NaN,NaN,0,NaN,1
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,0,2011-01-29,NaN,NaN,NaN,NaN,0,NaN,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,0,2011-01-29,NaN,NaN,NaN,NaN,0,NaN,1
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,0,2011-01-29,NaN,NaN,NaN,NaN,0,NaN,1


In [17]:
new_CA_1.to_csv("/Users/apple/Downloads/new_CA_1.csv", index=False)

#### *Final Data Overview*

In [20]:
# Columns in CA_1
new_CA_1.columns

Index(['item_id', 'dept_id', 'cat_id', 'sales', 'date', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'sell_price',
       'day_int'],
      dtype='object')

- **Scope**: All analysis and forecasting will be performed exclusively for the CA_1 store.
- **Target Variable**: The sales column represents the daily unit demand for each item.
- **Temporal Aspect**: date and day_int are critical for time-series analysis.
- **Hierarchical Structure**: The data has a clear hierarchy: Store (CA_1) -> Category (cat_id) -> Department (dept_id) -> Item (item_id). There are 3 main categories (FOODS, HOBBIES, HOUSEHOLD), with multiple departments nested within each.

In [21]:
# Basic information of CA_1
new_CA_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832737 entries, 0 to 5832736
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   item_id       object 
 1   dept_id       object 
 2   cat_id        object 
 3   sales         int64  
 4   date          object 
 5   event_name_1  object 
 6   event_type_1  object 
 7   event_name_2  object 
 8   event_type_2  object 
 9   snap_CA       int8   
 10  sell_price    float16
 11  day_int       int64  
dtypes: float16(1), int64(2), int8(1), object(8)
memory usage: 461.7+ MB


### **2.2. Data Preprocessing & Cleaning**

#### *Handling missing values*

In [22]:
# Fill NaN by 'No Event' in the event columns
new_CA_1[["event_name_1", "event_type_1", "event_name_2", "event_type_2"]] = new_CA_1[["event_name_1", "event_type_1", "event_name_2", "event_type_2"]].fillna('No Event')
new_CA_1

,item_id,dept_id,cat_id,sales,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,day_int
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,0,2011-01-29,No Event,No Event,No Event,No Event,0,NaN,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,0,2011-01-29,No Event,No Event,No Event,No Event,0,NaN,1
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,0,2011-01-29,No Event,No Event,No Event,No Event,0,NaN,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,0,2011-01-29,No Event,No Event,No Event,No Event,0,NaN,1
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,0,2011-01-29,No Event,No Event,No Event,No Event,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5832732,FOODS_3_823,FOODS_3,FOODS,1,2016-04-24,No Event,No Event,No Event,No Event,0,2.980469,1913
5832733,FOODS_3_824,FOODS_3,FOODS,0,2016-04-24,No Event,No Event,No Event,No Event,0,2.480469,1913
5832734,FOODS_3_825,FOODS_3,FOODS,2,2016-04-24,No Event,No Event,No Event,No Event,0,3.980469,1913
5832735,FOODS_3_826,FOODS_3,FOODS,0,2016-04-24,No Event,No Event,No Event,No Event,0,1.280273,1913


In [23]:
# Handle NaN in sell_price: Forward-fill and add a binary indicator flag
# This is crucial for your project, as discussed.
# Sort by item_id and date first to ensure correct ffill for each item's time series
new_CA_1 = new_CA_1.sort_values(by=['item_id', 'date']).reset_index(drop=True)

# Create a flag for when price was originally missing
new_CA_1['is_price_missing'] = new_CA_1['sell_price'].isna().astype(int)

# Forward-fill NaN sell_price values within each item_id group
new_CA_1['sell_price'] = new_CA_1.groupby('item_id')['sell_price'].ffill()

# Some items might have NaNs at the beginning of their series if they never had a price until later
# For these, I will backfill (bfill) or fill with a default value (e.g., 0, or mean of that item's prices)
# If a product was never sold with a price in the entire history, ffill might leave NaNs at the start.
# Let's check remaining NaNs and consider a fill strategy for them.
remaining_price_na = new_CA_1['sell_price'].isna().sum()
if remaining_price_na > 0:
    print(f"{remaining_price_na} NaN values remain in 'sell_price' after ffill. These are likely at the start of some item series.")
    # Option: Backfill these remaining NaNs for robust imputation
    new_CA_1['sell_price'] = new_CA_1.groupby('item_id')['sell_price'].bfill()
    remaining_price_na = new_CA_1['sell_price'].isna().sum()
    # If still NaN, fill with 0
    if remaining_price_na > 0:
        print(f"Warning: {remaining_price_na} NaN values still remain after bfill. These items might have no price history at all. Filling with 0.")
        new_CA_1['sell_price'] = new_CA_1['sell_price'].fillna(0)

print("Handled NaN values in 'sell_price' (ffill + bfill for remaining, then 0 for any absolute missing).")

1129842 NaN values remain in 'sell_price' after ffill. These are likely at the start of some item series.
Handled NaN values in 'sell_price' (ffill + bfill for remaining, then 0 for any absolute missing).


In [24]:
# Convert categorical columns to 'category' dtype for memory efficiency
categorical_cols = ['item_id', 'dept_id', 'cat_id',
                    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for col in categorical_cols:
    if col in new_CA_1.columns:
        new_CA_1[col] = new_CA_1[col].astype('category')
        
 # Convert date column to 'datetime' dtype     
new_CA_1['date'] = pd.to_datetime(new_CA_1['date'])

print(new_CA_1.info())
print(new_CA_1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832737 entries, 0 to 5832736
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   item_id           category      
 1   dept_id           category      
 2   cat_id            category      
 3   sales             int64         
 4   date              datetime64[ns]
 5   event_name_1      category      
 6   event_type_1      category      
 7   event_name_2      category      
 8   event_type_2      category      
 9   snap_CA           int8          
 10  sell_price        float16       
 11  day_int           int64         
 12  is_price_missing  int64         
dtypes: category(7), datetime64[ns](1), float16(1), int64(3), int8(1)
memory usage: 239.3 MB
None
       item_id  dept_id cat_id  sales       date event_name_1 event_type_1  \
0  FOODS_1_001  FOODS_1  FOODS      3 2011-01-29     No Event     No Event   
1  FOODS_1_001  FOODS_1  FOODS      0 2011-01-30     No Event  

## **III. EDA**

#### **3.1 Overall CA_1 Store Performance**

#### *Total Sales Trend and Decomposition*

In [25]:
# Sum sales of all categories by date
overall_sales = new_CA_1.groupby('date')['sales'].sum().reset_index()
overall_sales

,date,sales
0,2011-01-29,4337
1,2011-01-30,4155
2,2011-01-31,2816
3,2011-02-01,3051
4,2011-02-02,2630
...,...,...
1908,2016-04-20,3722
1909,2016-04-21,3709
1910,2016-04-22,4387
1911,2016-04-23,5577


In [26]:
from statsmodels.tsa.seasonal import STL

# Aggregate total daily sales for CA_1 store
sum_daily_sales = new_CA_1.groupby('date')['sales'].sum().sort_index()

# STL Decomposition for Weekly Seasonality (Period = 7)
stl_weekly = STL(sum_daily_sales, seasonal=13, trend=21, period=7, robust=True)
res_weekly = stl_weekly.fit()

# Visualize STL Decomposition (weekly)
fig = make_subplots(rows=4, cols=1,
        subplot_titles=(
            f'Original Data',
            f'Trend',
            f'Seasonal',
            f'Residual'
        ),
        vertical_spacing=0.1)

# Observed Component
fig.add_trace(go.Scatter(x=res_weekly.observed.index, y=res_weekly.observed,
                             mode='lines', name='Observed', line=dict(color='#007DC6')), row=1, col=1)

# Trend Component
fig.add_trace(go.Scatter(x=res_weekly.trend.index, y=res_weekly.trend,
                             mode='lines', name='Trend', line=dict(color='#007DC6')), row=2, col=1)

# Seasonal Component
fig.add_trace(go.Scatter(x=res_weekly.seasonal.index, y=res_weekly.seasonal,
                             mode='lines', name='Seasonal', line=dict(color='#4DBDF5')), row=3, col=1)

# Residual Component
fig.add_trace(go.Scatter(x=res_weekly.resid.index, y=res_weekly.resid,
                             mode='lines', name='Residuals', line=dict(color='#FFC220')), row=4, col=1)

fig.update_layout(title_font_size=24, title_font_color ='black',
        title_text=f'<b>STL Decomposition of Total CA_1 Sales (Weekly Seasonality)</b>', template='plotly_white',
        height=1200, width=1200,
        showlegend=False)
fig.show()

- Key Observations:
    + Strong Upward Trend: There's a clear and sustained upward trend in sales for CA_1 over the observed period (e.g., 2011-2016, as seen in the previous analysis). This signifies overall business growth or increasing market penetration.
    + Prominent Weekly Seasonality: The seasonal component will show consistent peaks and troughs repeating every 7 days. Typically, sales are higher on weekends and lower on weekdays. This is a highly predictable pattern.
    + Pronounced Yearly Seasonality: While there be can specified "weekly seasonality," the decomposition often reveals yearly patterns too (e.g., spikes during holidays, dips after New Year). If present, these would be evident as larger cycles within the seasonal component.
    + Non-Uniform Residuals: The residuals, even after accounting for trend and seasonality, might not be purely random. They could show occasional spikes or periods of increased variance.
- Insights for Demand Forecasting:
    + Feature Engineering Validation: This decomposition fundamentally justifies the creation of various time-based features: year or day_int for the trend, and day_of_week, month, week_of_year, day_of_year, is_weekend for seasonality.
    + Model Complexity: The presence of both strong trend and seasonality indicates that a simple averaging model won't suffice. The model needs to explicitly capture these patterns.
    + Understanding Residuals: The residual component is where the unexplainable variability lies. Minimizing this error is the model's goal. It also indicates where external factors (events, promotions, supply chain issues) or inherent randomness dominate.

#### *Anomaly Detection*

In [27]:
# Anomaly Detection
resisual = res_weekly.resid
resisual_mean = resisual.mean()
resisual_std = resisual.std()

# Cap residuals
lower = resisual_mean - 3 * resisual_std
upper = resisual_mean + 3 * resisual_std

# Visualize residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=resisual.index, y=resisual,
                             mode='lines', name='Residuals', line=dict(color='#FFC220')))
fig.add_shape(type='line', x0=resisual.index.min(),  x1=resisual.index.max(),  
    y0=lower,y1=lower,
    line=dict(color='royalblue', dash='solid'),
    name='Lower Threshold'
)
fig.add_shape(type='line', x0=resisual.index.min(),  x1=resisual.index.max(),  
    y0=upper,y1=upper,
    line=dict(color='royalblue', dash='solid'),
    name='Lower Threshold'
)
fig.update_layout(template='plotly_white', title='Anomaly Detection', title_font_size=24)
fig.show()


In [28]:
anomalies = sum_daily_sales[(resisual > upper) | (resisual < lower)]
anomalies

date
2011-11-26    3077
2011-12-25       0
2012-01-01    2648
2012-04-08    3962
2012-09-03    5157
2012-11-21    3843
2012-11-24    2964
2012-12-25       0
2013-02-02    6232
2013-07-03    5295
2013-08-25    6948
2013-09-02    5838
2013-11-28    2359
2013-11-29    2635
2013-11-30    3452
2013-12-25       0
2014-02-01    6511
2014-05-11    4184
2014-07-03    5582
2014-11-26    4434
2014-12-11    2298
2014-12-25       0
2015-04-03    5632
2015-07-03    6369
2015-07-04    4434
2015-09-07    5636
2015-11-26    2160
2015-11-27    2696
2015-11-28    3812
2015-12-25       0
2015-12-26    4137
2015-12-27    4010
2016-03-27    4669
Name: sales, dtype: int64

Unmodeled Events/Outliers: Significant spikes or drops in the residuals often correspond to events that were not explicitly included in the seasonality or trend components, or whose impact was not fully captured. 

- Insights for Demand Forecasting:
    + Feature Engineering Opportunities: Analyzing the residuals is a critical step for further feature engineering. If you see consistent patterns (e.g., spikes on specific dates, or periods of higher variance), it suggests that new features are needed to capture these explanations. Examples:
    + External Events: More granular event data, local holidays, school breaks.
    + Promotional Calendar: Information about specific store promotions.
    + Price Changes: If sell_price_change doesn't fully capture the impact, more sophisticated price elasticity features might be needed.
    + Supply Chain Flags: Features indicating inventory levels or out-of-stock situations.
    + Error Analysis: The residuals provide a direct visual of where the model is struggling. Large residuals (positive or negative) point to periods of significant under- or over-prediction.
Model Limitations: Even with robust feature engineering, some residuals are unavoidable due to inherent randomness. This sets the lower bound on achievable forecast error.

#### **3.2 Category (cat_id) Level Analysis - Sales Contribution**

#### *Total Sales by cat_id*

In [29]:
# Aggregate total sales by category
category_sales = new_CA_1.groupby('cat_id')['sales'].sum().sort_values(ascending=False).reset_index()

# Visualize total sales by category
fig = go.Figure()
fig.add_trace(go.Bar(x=category_sales['cat_id'], y=category_sales['sales'], marker_color='#007DC6', text=category_sales['sales'],
        textposition="auto"))
fig.update_layout(template='plotly_white',  title='Total Sales by Category in CA_1', title_font_size=24, 
                   yaxis_title="Total Sales",  xaxis_title="Category", height=500, width=900)
fig.show()

- Key Observations:
    + FOODS Domination: FOODS is by far the largest category in terms of total sales, dwarfing HOUSEHOLD and HOBBIES.
    + Tiered Contribution: HOUSEHOLD is the second-largest, and HOBBIES contributes the least to the overall sales of CA_1.
- Insights for Demand Forecasting:
    + High-Level Feature Importance: cat_id is a crucial categorical feature for the model to capture the overall scale and behavior of these broad product groups.
    + Strategic Focus: This re-emphasizes that forecasting FOODS category sales accurately is paramount due to its overwhelming contribution to total revenue. Error in FOODS forecasts will have the largest impact on overall store forecasts.

#### *Cat-specific trends*

In [30]:
# Aggregate sales by category over time
daily_cat_sales = new_CA_1.groupby(['date', 'cat_id'])['sales'].sum().reset_index()
print(daily_cat_sales)

           date     cat_id  sales
0    2011-01-29      FOODS   3239
1    2011-01-29    HOBBIES    556
2    2011-01-29  HOUSEHOLD    542
3    2011-01-30      FOODS   3137
4    2011-01-30    HOBBIES    498
...         ...        ...    ...
5734 2016-04-23    HOBBIES    670
5735 2016-04-23  HOUSEHOLD   1252
5736 2016-04-24      FOODS   4053
5737 2016-04-24    HOBBIES    714
5738 2016-04-24  HOUSEHOLD   1346

[5739 rows x 3 columns]


In [31]:
# Visualize sales by category over time
fig = go.Figure()
fig = px.line(daily_cat_sales, x='date', y='sales', color='cat_id',
    title='Daily Sales Trends by Category in CA_1')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title_font_size=24, template='plotly_white', yaxis_title='Sales', xaxis_title='Years')
fig.show()

- Key Observations:
    + Category-Level Trends: FOODS would show the highest daily sales, followed by HOUSEHOLD, and then HOBBIES.
    + Shared Seasonality: All categories would likely exhibit similar weekly and yearly seasonality patterns as the overall store sales, perhaps with slight variations in amplitude.
    + Response to Major Events: Broader events (like major holidays) would likely cause synchronized spikes across all categories, though some categories (e.g., FOODS) might react more strongly to food-related holidays.
- Insights for Demand Forecasting:
    + Validation of cat_id Feature: Confirms cat_id as another important categorical feature, capturing higher-level demand patterns.
    + Hierarchical Understanding: This level of aggregation helps understand how major events or long-term trends impact broad product groups.

#### **3.3 Department (dept_id) Level Analysis - Sales Contribution**

In [32]:
# Aggregate sales by department over time
dept_sales = new_CA_1.groupby('dept_id')['sales'].sum().sort_values(ascending=False).reset_index()
print(dept_sales)

       dept_id    sales
0      FOODS_3  3927835
1  HOUSEHOLD_1  1100165
2      FOODS_2   885144
3    HOBBIES_1   821524
4      FOODS_1   567849
5  HOUSEHOLD_2   340545
6    HOBBIES_2    55154


In [33]:
# Visualize total sales by department
fig = go.Figure()
fig.add_trace(go.Bar(x=dept_sales['dept_id'], y=dept_sales['sales'], marker_color='#007DC6', text=dept_sales['sales'],
        textposition="auto"))
fig.update_layout(template='plotly_white',  title='Total Sales by Department in CA_1', title_font_size=24, 
                   yaxis_title="Total Sales",  xaxis_title="Department", height=500, width=900)
fig.show()

- Key Observations:
    + FOODS_3 Dominance: FOODS_3 stands out as the department with significantly the highest total sales, accounting for a disproportionately large share of revenue within CA_1.
    + Varying Contributions: Other departments contribute much less
- Insights for Demand Forecasting:
    + Categorical Feature Importance: This visually confirms dept_id as a highly discriminative categorical feature. The model needs to assign different "weights" or baselines to each department.
    + Business Focus: For business stakeholders, this highlights that FOODS_3 is the most critical department to forecast accurately due to its significant impact on overall sales.
    + Scale Differences: The vast difference in scales between departments needs to be handled by the model. Tree-based models are generally robust to this, but it emphasizes the need for careful evaluation of error metrics across different departments.

In [34]:
# Aggregate sales by department over time
daily_dept_sales = new_CA_1.groupby(['date', 'dept_id'])['sales'].sum().reset_index()
print(daily_dept_sales)

# Visualize sales by department over time
fig = go.Figure()
fig = px.line(daily_dept_sales, x='date', y='sales', color='dept_id',
    title='Daily Sales Trends by Department in CA_1')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title_font_size=24, height=600,template='plotly_white', yaxis_title='Sales', xaxis_title='Years')
fig.show()

            date      dept_id  sales
0     2011-01-29      FOODS_1    297
1     2011-01-29      FOODS_2    674
2     2011-01-29      FOODS_3   2268
3     2011-01-29    HOBBIES_1    528
4     2011-01-29    HOBBIES_2     28
...          ...          ...    ...
13386 2016-04-24      FOODS_3   2990
13387 2016-04-24    HOBBIES_1    641
13388 2016-04-24    HOBBIES_2     73
13389 2016-04-24  HOUSEHOLD_1   1022
13390 2016-04-24  HOUSEHOLD_2    324

[13391 rows x 3 columns]


- Key Observations
    + Department-Specific Trends: Each department would likely exhibit its own sales volume and trend. FOODS_3 would show the highest sales.
    + Seasonality within Departments: Each department would likely follow the general weekly and yearly seasonality patterns observed in the overall sales, but potentially with varying magnitudes. For instance, HOBBIES might have stronger peaks around specific craft-related holidays.
    + Impact of Events: Department-specific spikes or dips corresponding to events would be visible.
- Insights for Demand Forecasting:
    + Department-Level Nuances: If available, this granular view would confirm that while overall patterns exist, each department has unique sales dynamics that the model must capture through dept_id as a categorical feature and its interactions with other features.
    + Validation of dept_id Feature: Reinforces the importance of dept_id as a distinct predictor.
    + Daily Granularity: Emphasizes that features should support daily predictions and capture daily fluctuations.

#### **3.4 Item (item_id) Level Variability - Coefficient of Variation (CV)**

In [35]:
# Calculate CV for each item_id
# Formula: CV = std/mu
item_demand_stats = new_CA_1.groupby('item_id')['sales'].agg(['mean', 'std']).reset_index()
item_demand_stats['cv'] = item_demand_stats['std'] / item_demand_stats['mean']
item_demand_stats = item_demand_stats.sort_values(by='cv', ascending=False)

# Display top 10 most volatile items
print("\nTop 10 Most Volatile Items (by CV):")
print(item_demand_stats.head(10))

# Visualize
fig= px.histogram(item_demand_stats, x ='cv', nbins=50, title='Distribution of Demand Coefficient of Variation (CV) per Item_ID in CA_1')
fig.update_layout(title_font_size=24, template='plotly_white', height=550, width=1000,
                  yaxis_title='Number of Items', xaxis_title='Coefficient of Variation (CV)')
fig.update_traces(marker_color ="#007DC6", marker_line_width=0.25, marker_line_color="lightgrey")
fig.show()


Top 10 Most Volatile Items (by CV):
              item_id      mean       std         cv
1462    HOBBIES_1_026  0.010978  0.118325  10.778830
2372  HOUSEHOLD_1_378  0.010978  0.109127   9.940957
2130  HOUSEHOLD_1_133  0.651856  5.725783   8.783819
962       FOODS_3_350  0.044956  0.352912   7.850243
1207      FOODS_3_595  0.042865  0.317289   7.402113
908       FOODS_3_296  0.029273  0.212527   7.260074
423       FOODS_2_209  0.040251  0.290992   7.229440
331       FOODS_2_117  0.047047  0.326454   6.938957
1875    HOBBIES_2_023  0.040251  0.276239   6.862920
2295  HOUSEHOLD_1_300  0.029796  0.201071   6.748239


- Key Observations:
    + High Volatility for Many Items: The histogram is heavily skewed to the right, with a large concentration of items showing a high CV (often > 1, or even > 2). This indicates that for a significant number of products, their standard deviation of sales is greater than or equal to their average sales.
    + Intermittent Demand: High CV values often point to intermittent demand patterns, where sales are sporadic, with many zero-sales days interspersed with occasional sales.
    + Few Stable Items: Only a small proportion of items exhibit low CV values, suggesting relatively stable and predictable demand.
- Insights for Demand Forecasting:
    + Forecasting Difficulty: This is a crucial insight. Forecasting items with high CV is inherently more difficult. Standard regression models might struggle with high proportions of zero sales and high variance. Thus, I plan to use strong regressors like RF, XBG with Ramdomized Search CV for hyperparameter tunning.
    + Importance of Target Transformation: The need for log1p transformation of demand is reinforced. It helps to reduce the impact of these extreme values and make the distribution more symmetrical for the model. *(In the below model part, I have not transform y, but I plan to perform 2 times modelling, one with transoformed y and aother without transformation to see which one is better; if I would have enough computational capability)*
    + Volatility Features: The demand_roll_std features (rolling standard deviation) are essential. They provide the model with direct information about the recent variability of an item's sales, allowing it to potentially adjust its predictions or uncertainty estimates.
    + Error Metrics: Given the high volatility, metrics like Mean Absolute Error (MAE) might be preferred over Root Mean Squared Error (RMSE), as RMSE heavily penalizes large errors, which are more likely for highly volatile items.

## **IV. Feature Engineering**

In [36]:
from sklearn.model_selection import TimeSeriesSplit

For the later forcasting, I will assume that sales in the dataset is the demand of it.

In [37]:
# Rename target column from 'sales' to 'demand'
new_CA_1 = new_CA_1.rename(columns={'sales': 'demand'})

# Ensure the dataset is sorted by 'item_id' and 'date' for later lag & rolling calculations
new_CA_1 = new_CA_1.sort_values(by=['item_id', 'date']).reset_index(drop=True)
new_CA_1.head(10)

,item_id,dept_id,cat_id,demand,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,day_int,is_price_missing
0,FOODS_1_001,FOODS_1,FOODS,3,2011-01-29,No Event,No Event,No Event,No Event,0,2.0,1,0
1,FOODS_1_001,FOODS_1,FOODS,0,2011-01-30,No Event,No Event,No Event,No Event,0,2.0,2,0
2,FOODS_1_001,FOODS_1,FOODS,0,2011-01-31,No Event,No Event,No Event,No Event,0,2.0,3,0
3,FOODS_1_001,FOODS_1,FOODS,1,2011-02-01,No Event,No Event,No Event,No Event,1,2.0,4,0
4,FOODS_1_001,FOODS_1,FOODS,4,2011-02-02,No Event,No Event,No Event,No Event,1,2.0,5,0
5,FOODS_1_001,FOODS_1,FOODS,2,2011-02-03,No Event,No Event,No Event,No Event,1,2.0,6,0
6,FOODS_1_001,FOODS_1,FOODS,0,2011-02-04,No Event,No Event,No Event,No Event,1,2.0,7,0
7,FOODS_1_001,FOODS_1,FOODS,2,2011-02-05,No Event,No Event,No Event,No Event,1,2.0,8,0
8,FOODS_1_001,FOODS_1,FOODS,0,2011-02-06,SuperBowl,Sporting,No Event,No Event,1,2.0,9,0
9,FOODS_1_001,FOODS_1,FOODS,0,2011-02-07,No Event,No Event,No Event,No Event,1,2.0,10,0


### **4.1 Feature Extraction**

### *Time-Based Features*

In [38]:
import datetime as dt

In [39]:
# These features directly encode seasonality and trend.
new_CA_1['day_of_week'] = new_CA_1['date'].dt.dayofweek # Monday=0, Sunday=6
new_CA_1['day_of_month'] = new_CA_1['date'].dt.day
new_CA_1['week_of_year'] = new_CA_1['date'].dt.isocalendar().week.astype(int) # Use isocalendar for week number
new_CA_1['month'] = new_CA_1['date'].dt.month
new_CA_1['year'] = new_CA_1['date'].dt.year
new_CA_1['quarter'] = new_CA_1['date'].dt.quarter
new_CA_1['is_weekend'] = (new_CA_1['day_of_week'] >= 5).astype(int) # 0 for weekday, 1 for weekend
new_CA_1['day_of_year'] = new_CA_1['date'].dt.dayofyear
new_CA_1

,item_id,dept_id,cat_id,demand,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,day_int,is_price_missing,day_of_week,day_of_month,week_of_year,month,year,quarter,is_weekend,day_of_year
0,FOODS_1_001,FOODS_1,FOODS,3,2011-01-29,No Event,No Event,No Event,No Event,0,...,1,0,5,29,4,1,2011,1,1,29
1,FOODS_1_001,FOODS_1,FOODS,0,2011-01-30,No Event,No Event,No Event,No Event,0,...,2,0,6,30,4,1,2011,1,1,30
2,FOODS_1_001,FOODS_1,FOODS,0,2011-01-31,No Event,No Event,No Event,No Event,0,...,3,0,0,31,5,1,2011,1,0,31
3,FOODS_1_001,FOODS_1,FOODS,1,2011-02-01,No Event,No Event,No Event,No Event,1,...,4,0,1,1,5,2,2011,1,0,32
4,FOODS_1_001,FOODS_1,FOODS,4,2011-02-02,No Event,No Event,No Event,No Event,1,...,5,0,2,2,5,2,2011,1,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832732,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,0,2016-04-20,No Event,No Event,No Event,No Event,0,...,1909,0,2,20,16,4,2016,2,0,111
5832733,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,1,2016-04-21,No Event,No Event,No Event,No Event,0,...,1910,0,3,21,16,4,2016,2,0,112
5832734,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,1,2016-04-22,No Event,No Event,No Event,No Event,0,...,1911,0,4,22,16,4,2016,2,0,113
5832735,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,0,2016-04-23,No Event,No Event,No Event,No Event,0,...,1912,0,5,23,16,4,2016,2,1,114


### **4.2 Feature Transformation**


In [40]:
# Define the list of categorical columns to encode
categorical_cols_to_encode = [
    'item_id', 'dept_id', 'cat_id','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

# Create a dictionary to store LabelEncoders for potential inverse transformation later
label_encoders = {}

for col in categorical_cols_to_encode:
    if col in new_CA_1.columns:
        print(f"Encoding column: {col}")
        le = LabelEncoder()
        new_CA_1[col] = le.fit_transform(new_CA_1[col]) 
        label_encoders[col] = le # Store the fitted encoder


print(new_CA_1[['item_id', 'dept_id', 'cat_id', 'event_name_1', 'event_type_1']].head())
print("Dtypes after encoding:")
print(new_CA_1[['item_id', 'dept_id', 'cat_id', 'event_name_1', 'event_type_1']].dtypes)

Encoding column: item_id
Encoding column: dept_id
Encoding column: cat_id
Encoding column: event_name_1
Encoding column: event_type_1
Encoding column: event_name_2
Encoding column: event_type_2
   item_id  dept_id  cat_id  event_name_1  event_type_1
0        0        0       0            19             2
1        0        0       0            19             2
2        0        0       0            19             2
3        0        0       0            19             2
4        0        0       0            19             2
Dtypes after encoding:
item_id         int64
dept_id         int64
cat_id          int64
event_name_1    int64
event_type_1    int64
dtype: object


## **5. Modelling**

In [41]:
# Define X and y
y = new_CA_1['demand']
X =  new_CA_1.copy()
X = X.drop(['demand','date'], axis=1)
dates_for_split = new_CA_1['date'] # Used for TimeSeriesSplit

# TimeSeriesSplit Setup
# n_splits: Number of cross-validation folds.
# test_size: The number of samples (days) in each validation set.
# gap: The number of samples (days) to exclude between the training and validation sets to simulate real-world lag.
tscv = TimeSeriesSplit(n_splits=3, test_size=28*2, gap=28) # 3 folds, 2 months validation, 1 month gap

# Define scoring metric for RandomizedSearchCV
# 'neg_mean_absolute_error' is typically good for demand forecasting (minimizing MAE)
# 'neg_root_mean_squared_error' is for RMSE
scoring_metric = 'neg_mean_absolute_error'
print(f"\nTimeSeriesSplit configured with n_splits={tscv.n_splits}, test_size={tscv.test_size}, gap={tscv.gap}")
print(f"RandomizedSearchCV will use '{scoring_metric}' as scoring metric.")


TimeSeriesSplit configured with n_splits=3, test_size=56, gap=28
RandomizedSearchCV will use 'neg_mean_absolute_error' as scoring metric.


### *XGBoost Regressor*

In [ ]:
import xgboost as xgb
# Define model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='mae', # Metric to monitor during training
    random_state=42,
    n_jobs=-1 # Use all available CPU cores
)

# Hyper-tuning with RandomizedSearchCV
xgb_param_dist = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.01, 0.1],
    'max_depth': [6, 8, 10],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_dist,
    n_iter=5,
    scoring=scoring_metric,
    cv=tscv,
    random_state=42,
    n_jobs=-1
)

xgb_random_search.fit(X, y)

print("\nXGBoostRegressor Best Parameters:", xgb_random_search.best_params_)
print("XGBoostRegressor Best MAE:", xgb_random_search.best_score_)

### **5.2 Evaluation**

In [ ]:
# Make predictions on the last validation fold of the TimeSeriesSplit for demonstration
print("\nEvaluating the best XGBoost model on the last validation fold:")
train_idx, val_idx = list(tscv.split(X, y, dates_for_split))[-1] # Get the last fold

X_val_final = X.iloc[val_idx]
y_val_final = y.iloc[val_idx]

final_val_pred = xgb_random_search.best_estimator_.predict(X_val_final)


rmse = np.sqrt(mean_squared_error(y_val_final, final_val_pred))
mae = mean_absolute_error(y_val_final, final_val_pred)

print(f"XGBoost RMSE (last fold): {rmse:.4f}")
print(f"XGBoost MAE (last fold): {mae:.4f}")
